# Lab 2: Step Response of an RC Circuit

## Introduction

This lab has three primary learning objectives:

1. helping students learn how to perform real-time dynamic systems and
   control experiments 
   - using a Raspberry Pi and Arduino together with i$^2$c
   - using Dr. Krauss' `pybd_gui` and `py_block_diagram` modules
2. helping students learn about transfer functions through
   investigating the step response of an RC circuit
3. helping students learn to curve fit data using `scipy.optimize.fmin`

Throughout this course, we will use Arduino microcontrollers in conjunction with Python to run dynamic systems and control experiments.  

An RC circuit is a fairly simple dynamic system and will be used in this lab to help students begin to think in terms of transfer functions.  The results of an experimental step response will be used to estimate transfer function coefficients.  Experimental results will then be compared to Python simulations.


## Main Steps

- disconnect your Raspberry Pi from the cart/pendulum system and connect it to a monitor and keyboard 
- update the software on your Raspberry Pi and learn basic linux commands
- connect a Raspberry Pi to an Arduino using a level shifter and i$^2$c
- build an RC circuit and connect it to the Arduino
- learn to use `pybd_gui` to create a block diagram for your system and auto-generate the Raspberry Pi C++ code
- program the Arduino using the code provided
- overlay the experimental step response with the result of Laplace analysis "by hand" as well as simulation results using `control.step_response`
- curve fit the experimental step response data to improve your estimate of the TF parameters

## Part 1: Raspberry Pi prep, linux basics, and i$^2$c

Most of the Raspberry Pi's are still attached to a cart/pendulum system from the end of last year's class.  Remove the red shield so that you can unscrew the RPi from the cart.  You will need to connect the RPi to a monitor using a mini-HDMI connection (HDMI0).  You will also need a USB-C power supply as well as a mouse and keyboard.

Once you have a monitor, mouse, and keyboard attached to your RPi, power it on with the USB-C power supply.  Verify that it boots up to a desktop environment.

### i$^2$c Connections

A Raspberry Pi uses 3.3V while an Arduino Uno uses 5V.  **Never** connect any signal that can reach 5V directly to a Raspberry Pi.  Level shifters are custom circuits used to solve this problem.  The red shield contains a level shifter that connects to the i$^2$c pins on the Raspberry Pi:

<img src="https://drive.google.com/uc?id=18BOzmnBxxeEuUiEn_v1lI4IMmTro6XkG" width=500px>

i$^2$c is a communication protocol used between micro-controllers or other electronic devices.  The communication is mainly handled using one wire for data (`SDA`) and one for the clock (`SCL`).  A common ground (`GND`) is required and the level shifter needs 5V from the Arduino.  On an Arduino Uno, `SDA` is pin `A4` and `SCL` is `A5`.  You will also need to connect ground to your bread board for building your RC circuit.

## Linux Basics

You can sort of operate a Raspberry Pi using a mouse, but it will ultimately slow you down and limit your learning.  The terminal is your friend.  Open a terminal by clicking on the black rectangle icon near the Raspberry Pi logo in the top left of your screen.

A Jupyter notebook explaining [basic linux](basic_linux_commands.ipynb) commands is in the Lab 2 folder.  You can launch the Jupyter notebook server on a Raspberry Pi using the command `jupyter-notebook`.

You probably want to create a folder for your team to work in using the command `mkdir`.

## Software Update

At the start of nearly every lab, you will need to do at least the following two commands to get the most recent lab files:

`cd 345_lab_git`

`git pull origin main`

The first command changes the working directory to the folder `345_lab_git`.  The second command is the magical incantation to retrieve the latest code from the 345 lab cloud repository on `github.com`.

Most weeks you will also need to start by running a script that updates the lab software.  The command for this week is `sw_update_lab_2.sh`.

Keep in mind that you can use tab completion on most commands and you can also type part of a command and use the up arrow to search for a match in the command history.

Once you have run this week's update script, check the version of your install software using the command 

`check_pip_versions.py`

Dr. Krauss will tell you what the correct latest version numbers are.

## Treat `345_lab_git` as read only

This is how you get code from me.  Do your work in your own folder.  Use the command `cp` to copy files to your folder if you want to do that.

## Part 2: RC Step Response

### `pybd_gui` demo

**Key Ideas:**

- you have to create an actuator and a sensor before you can create a plant
    - we will use `i2c_actuator`, `i2c_sensor`, and `i2c_plant` for this week's lab
- creating a block is separate from placing it on the block diagram
    - you should have one absolute block placement and the rest should be relative
- you must specify which blocks or sensors print their output to the data file
    - if you accidentally skip this, the data file will contain only the loop count variable and the time in milliseconds
- when you are ready to generate the RPi code, you have to specify the output file path and then generate the code
    - the input template path should be set for you automatically

### compiling RPi C code

I have create a custom script that compiles and links the auto-generated C code (technically it is C++).  The script is called `rpidb_build.py` and it takes the name of your file as its only input:

`rpibd_build.py filename.c`

This should create a file called `filename.o` that is executable.  Run this code from the terminal using the command

`./filename.o`

where `./` just specifies that you mean the current folder.

## Arduino Code

In order for this lab to work, you have to make sure the Arduino is running the correct code.  The code should be in a folder called `arduino_i2c_siso_RC_circuit`.  You can launch the Arduino software on the RPi using the terminal command `arduino`.

## Step Response Testing

<img src="https://drive.google.com/uc?id=1RGNnAZyAhFgZZgN11-3Tfen_nabwZ6uQ" width=400px>

A schematic of an RC filter is shown above.
In order to run a step response test, you will need to modify the
Python code from Part 1 to communicate with the Arduino over serial.


## Saving Your Data

Once you have successfully run an RC step response test, save that data to a CSV file before you start curve fitting.  This will allow you to work on curve fitting without having to have your Arduino attached and running a new test each time.

The Raspberry Pi code should automatically save the most recent data to a file called `data.txt`.  Copy this file to your laptop to work on the rest of the lab (unless you want to curve fit on the Raspberry Pi).

You will know you have done this correctly if you can use `np.loadtxt` to load the data into Python and plot it like you did in lab 1.


## Curve Fitting

Once you have the step response data, use `scipy.optimize.fmin` to
curve fit it.  In order to do this, you will need to create a function that returns $v_{out}$ for a 5V step input.  First, find the transfer function of the circuit

$$G(s) = \frac{V_{out}(s)}{V_{in}(s)}$$

We found the transfer function in lecture.  Verify the transfer
function we found in lecture using Kirchoff's laws, Ohm's law, and the
voltage/current relationship of a capacitor.  Once you have verified
the transfer function, write a function that finds the step response
with a 5V input.  You ought to be able to modify code from lab 1 to
create the function you need for curve fitting.


## optimize.fmin Example

A Jupyter notebook example has been created to introduce curve fitting in Python based on the function `scipy.optimize.fmin`.  The example requires a `csv` data file.

The Jupyter notebook and csv file are in the lab 2 folder.

### Curve Fitting Tips

In order to fit a curve to the step response using `optimize.fmin`, you need a function that takes the unknown transfer function coefficients (i.e. $p$ in this case) and returns the step response.  This is very similar to what we did in lab 1 except that we need to find the step response at the same points in time where we have experimental data.  There are two options for doing this:

1. Solve the step response symbolically with $p$ as a variable and then just have your `mymodel` function use that symbolic expression
2. Pass in your time array as the second input to `control.step_response`

Option 2 is in some ways easier but comes with one additional
challenge: `control.step_response` expects the time vector to have
perfect time steps that are exactly the same.  Essentially, it is expecting that your time vector was created using `np.arange`.  So, you need to find a way to create a fake, perfect time vector that is as close as possible to your real, experimental time vector.  Find the average time step `dt` from your experimental time vector and use it along with `np.arange` to create a fake time vector that has the same length as the experimental time vector.

Note that in Python, functions always have read-only access to all of
the variables in the notebook or Python file where they reside.  So,
you need to pass your fake time vector as a second input to
`control.step_response`, but it does not need to be an input to the
`mymodel` function.  All you need to do is define the fake time vector
somewhere in the notebook.  The only input to `mymodel` should be a
list of unknown coefficients, `[p]`.


## Step Response Simulation

You are required to compare your experimental results to a simulation
of the step response using either `control.step_response` or
`control.forced_response` from the Python `control` module.  You will
need the transfer function of the system to do this.  To show your
results, overlay data from your experimental step response on the same
plot as simulation results based on your optimized coefficient values.
Also, verify that the step response from the `control` module agrees with
your derivation for $v_{out}(t)$ for the step input.


# Comprehension Questions

### CQ1: step response

What steps are necessary to perform step response analysis "by hand"?  How do you do the same steps using Python to verify your answer?

### CQ2: optimize.fmin

What does `optimze.fmin` actually do?  What is the scalar value returned by the cost function?  